## 스트림릿 : 수집하기 버튼

## 회사_올리브영
- 직무 : 
- 직무 소개 :
- 필요성 :

- 수집 계획 및 내용:

아모레퍼시픽 사이트도 수집, 왜 하필 아모레퍼시픽 사진?

# 올리브영_스킨 제품 수집(판매순)

In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

def get_multiple_page(save_path='올영 스킨,토너 판매순.데이터xlsx'):
    # Chrome WebDriver 시작
    browser = webdriver.Chrome()
    results = []  # 결과 저장용 리스트
    
    try:
        # 1부터 10페이지까지 정보 수집 
        for page_idx in range(1, 6):
            # URL 변경
            url = f'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010013&fltDispCatNo=&prdSort=03&pageIdx={page_idx}'
            browser.get(url)
            time.sleep(2)  # 페이지 로딩 대기
            
            # 상품 목록 가져오기
            product_elements = browser.find_elements(By.CSS_SELECTOR, 'div.prd_info > a')
            num_products = len(product_elements)  # 상품 개수 확인
            
            # 상품별 데이터 수집
            for product_idx in range(num_products):
                # 상품 클릭
                product_elements = browser.find_elements(By.CSS_SELECTOR, 'div.prd_info > a')
                product_elements[product_idx].click()
                time.sleep(2)  # 상세 페이지 로딩 대기
                
                soup = BeautifulSoup(browser.page_source, 'html.parser')
                
                # 상품 정보 추출
                brand = soup.select_one('p.prd_brand').text.strip() if soup.select_one('p.prd_brand') else None
                product_name = soup.select_one('.prd_name').text.strip() if soup.select_one('.prd_name') else None
                price = soup.select_one('div.price > .price-2 > strong').text.strip() if soup.select_one('div.price > .price-2 > strong') else None
                rating = soup.select_one('#repReview > b').text.strip() if soup.select_one('#repReview > b') else None
                
                # 구성 정보 추출
                go_composition = browser.find_elements(By.CSS_SELECTOR, 'li#buyInfo > a')
                composition_data = None
                if go_composition:
                    go_composition[0].click()
                    time.sleep(2)
                    
                    soup = BeautifulSoup(browser.page_source, 'html.parser')
                    composition_elements = soup.select('div#artcInfo > dl.detail_info_list > dd')
                    if composition_elements and len(composition_elements) > 6:
                        composition_data = composition_elements[6].text.strip()
                
                # 리뷰 데이터 추출
                skin_type_data = {}
                reviews = []
                go_review = browser.find_elements(By.CSS_SELECTOR, 'li#reviewInfo')
                if go_review:
                    go_review[0].click()
                    time.sleep(2)
                    
                    soup = BeautifulSoup(browser.page_source, 'html.parser')
                    for tag in soup.select('ul.list > li'):
                        label = tag.select_one('span.txt').text.strip() if tag.select_one('span.txt') else None
                        percentage = tag.select_one('em').text.strip() if tag.select_one('em') else None
                        if label and percentage:
                            skin_type_data[label] = percentage
                    
                    # 리뷰 데이터 페이지 순회
                    for review_page in range(1, 11):
                        review_elements = browser.find_elements(By.CSS_SELECTOR, 'div.txt_inner')
                        for review_element in review_elements:
                            reviews.append(review_element.text.strip())
                        
                        # 다음 페이지 버튼 클릭
                        try:
                            next_page_btn = browser.find_element(By.XPATH, f"//a[@data-page-no='{review_page + 1}']")
                            browser.execute_script("arguments[0].click();", next_page_btn)
                            time.sleep(2)
                        except:
                            break
                
                # 데이터 저장
                results.append({
                    "브랜드": brand,
                    "상품명": product_name,
                    "가격": price,
                    "별점": rating,
                    "구성정보": composition_data,
                    "피부타입": skin_type_data,
                    "리뷰": reviews
                })
                
                # 이전 페이지로 돌아가기
                browser.back()
                time.sleep(2)
    
    finally:
        browser.quit()  # 브라우저 종료
    
    # 결과 DataFrame 생성
    final_df = pd.DataFrame(results)
    
    # 엑셀로 저장
    final_df.to_excel(save_path, index=False, encoding='utf-8-sig')
    print(f"데이터가 엑셀 파일로 저장되었습니다: {save_path}")
    return final_df

In [39]:
get_multiple_page(save_path='올영 스킨,토너 판매순.xlsx')

데이터가 엑셀 파일로 저장되었습니다: 올영 스킨,토너 판매순.xlsx


,브랜드,상품명,가격,별점,구성정보,피부타입,리뷰
0,토리든,[단독기획] 토리든 다이브인 저분자 히알루론산 토너 300ml 기획(+100ml 추...,"15,700",4.8,"정제수, 부틸렌글라이콜, 다이프로필렌글라이콜, 1,2-헥산다이올, 글리세린, 베타인...","{'건성에 좋아요': '29%', '복합성에 좋아요': '55%', '지성에 좋아요...","[무난히 사용하기 좋은 토너에요 몇번 레이어드해서 바르면 좋아요, 🚨🚨토리든 토너 ..."
1,라운드랩,[1등토너] 라운드랩 1025 독도 토너 500ml 대용량 기획 (+100ml 증정),"27,000",4.8,"정제수, 부틸렌글라이콜, 글리세린, 펜틸렌글라이콜, 프로판다이올, 아이리쉬모스추출물...","{'건성에 좋아요': '23%', '복합성에 좋아요': '62%', '지성에 좋아요...","[피부진단해보니 건성이라는거에요.,\n수부지인줄알았눈데.,..\n그래서 스킨 레이어..."
2,더랩바이블랑두,[속보습] 더랩바이블랑두 올리고 히알루론산 딥 토너 500ml 대용량 기획 (+10...,"28,900",4.9,"정제수, 부틸렌글라이콜, 1,2-헥산다이올, 글리세린, 베타-글루칸, 베타인, 소듐...","{'건성에 좋아요': '40%', '복합성에 좋아요': '48%', '지성에 좋아요...","[순한 토너 중에 더랩바이블랑두 토너가 젤 좋아요 대용량 구메했더니 든든해요, 엄청..."
3,라네즈,[한정기획][미스트펌프 증정] 라네즈 크림스킨 170ml 기획 (+130ml 리필+...,"26,400",4.8,"정제수, 부틸렌글라이콜, 글리세린, 메도우폼씨오일, 아이소펜틸다이올, 1,2-헥산다...","{'건성에 좋아요': '54%', '복합성에 좋아요': '42%', '지성에 좋아요...",[🫶전 올영 메이트의 찐 간증 후기🫶\n\n사실 크림 스킨은 좋다는 말은 정말 많이...
4,토리든,[단독기획] 토리든 다이브인 저분자 히알루론산 토너 500ml 기획 (+화장솜 60...,"21,750",4.8,"1. 다이브인 저분자 히알루론산 토너 500ml (본품)정제수, 부틸렌글라이콜, 다...","{'건성에 좋아요': '33%', '복합성에 좋아요': '53%', '지성에 좋아요...","[무난히 사용하기 좋은 토너에요 몇번 레이어드해서 바르면 좋아요, 🚨🚨토리든 토너 ..."
...,...,...,...,...,...,...,...
115,싸이닉,싸이닉 더 심플 카밍 토너 145ml,"8,400",4.7,"정제수, 글리세린, 프로판다이올, 1,2-헥산다이올, 글리세레스-26, 펜틸렌글라이...","{'건성에 좋아요': '20%', '복합성에 좋아요': '60%', '지성에 좋아요...","[가격대비 양이 넉넉해요 토너패드 만들어서 아침에 가볍게 쓰고 잇어요, 후기좋아서 ..."
116,싸이닉,싸이닉 더 심플 카밍 토너 145ml,"8,400",4.7,"정제수, 글리세린, 프로판다이올, 1,2-헥산다이올, 글리세레스-26, 펜틸렌글라이...","{'건성에 좋아요': '20%', '복합성에 좋아요': '62%', '지성에 좋아요...","[몇년째 이것만 써요\n피부 깊숙히 침투하는 느낌? 부드럽게 흡수돼요, 그냥 편하게..."
117,클럽맨피노드,클럽맨리저브 젠트진 에프터쉐이브 50ml,"9,000",4.7,"변성알코올, 정제수, 향료, 벤질벤조에이트, 시트랄, 시트로넬올, 쿠마린, 유제놀,...","{'건성에 좋아요': '17%', '복합성에 좋아요': '67%', '지성에 좋아요...",[젠트진 향을 극찬하는 분들도 있길래\n큰 용량이 품절임에도 써봐야겠다싶어\n작은 ...
118,클럽맨피노드,클럽맨리저브 젠트진 에프터쉐이브 177ml,"20,000",4.8,"변성알코올, 정제수, 향료, 벤질벤조에이트, 시트랄, 시트로넬올, 쿠마린, 유제놀,...","{'건성에 좋아요': '12%', '복합성에 좋아요': '69%', '지성에 좋아요...",[면도 후 사용하면 엄청 따갑다고 하네요. 그리고 향은 엄청 진한 아저시 스킨 향이...


In [41]:
    import pandas as pd

    # 데이터 읽어오기 (예: 엑셀 파일)
    df = pd.read_excel('./올영 스킨,토너 판매순.xlsx')

    # 피부타입 추천 로직 정의
    def recommend_skin_type(row):
        try:
            # '피부타입' 컬럼 문자열을 딕셔너리로 변환
            skin_type_data = eval(row['피부타입'])  # 딕셔너리로 변환

            # 비교 대상 필터링: '건성에 좋아요', '복합성에 좋아요', '지성에 좋아요'
            relevant_data = {k: int(v.strip('%')) for k, v in skin_type_data.items() if k in ['건성에 좋아요', '복합성에 좋아요', '지성에 좋아요']}

            # 가장 높은 퍼센트의 키 찾기
            recommended_type = max(relevant_data, key=relevant_data.get)
            return recommended_type
        except Exception as e:
            return None  # 오류가 있으면 None 반환

    # 새 컬럼 추가
    df['피부타입추천'] = df.apply(recommend_skin_type, axis=1)

    # 결과를 엑셀로 저장
    df.to_excel('올영_스킨토너_판매순_스킨타입별.xlsx', index=False, encoding='utf-8-sig')

    print("피부타입 추천 컬럼이 추가된 결과가 저장되었습니다.")

피부타입 추천 컬럼이 추가된 결과가 저장되었습니다.
